In [ ]:
import numpy as np

from deep_orderbook.learn.test_learn import train_and_predict
from deep_orderbook.visu import Visualizer
from deep_orderbook.config import TrainConfig, ReplayConfig, ShaperConfig


vis = Visualizer()

train_config = TrainConfig(
    num_workers=10,
    batch_size=32,
    data_queue_size=512,
    num_levels=8,
    learning_rate=0.001,
)
replay_config = ReplayConfig(
    markets=["ETH-USD"],#, "BTC-USD", "ETH-BTC"],
    date_regexp='2024-1*T14',
    data_dir='/media/photoDS216/crypto/',
    every="500ms",
)
shaper_config = ShaperConfig(
    # rolling_window_size=4096,
    only_full_arrays=False,
    zoom_frac = 0.004,
    look_ahead = 16,
    look_ahead_side_bips = 32,
)
test_config = replay_config.but(date_regexp='2024-11-06T01')

# Define your asynchronous function to update the figure
async def online_learning(train_config, replay_config, shaper_config):
    async for books_arr, t2l, pxar, prediction, loss in train_and_predict(config=train_config, replay_config=replay_config, shaper_config=shaper_config, test_config=test_config):
        pred_t2l = prediction.reshape(t2l.shape).transpose(1, 0, 2)
        pred_t2l = np.clip(pred_t2l[:, :, 0], -1, 1)

        vis.add_loss(loss)
        vis.update(books_z_data=books_arr, level_reach_z_data=t2l, bidask=pxar, pred_t2l=pred_t2l)

await online_learning(train_config=train_config, replay_config=replay_config, shaper_config=shaper_config)